# Objective and Goal

1. The objective and goal for this project is to identify patterns and differences in characteristics between customers who have churned and those who 

haven't by running Analysis on a Telecommunication churn dataset. The insights i shall gain will be beneficial to the management as they will use 

these insights to develop strategies to retain customers and reduce he rate of churn among them.






2. The second objective and goal for this project is to develop a predictive model which will learn from the patterns in this dataset thereby gain the ability to 

predicts weather a customer is going to churn or not condsidering their interactions with services provided by the company which will assist the management 

identify and address customer concerns enhancing customer retention rates.

In [ ]:
#importing the necessary labraries needed for this project
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set()
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split,RandomizedSearchCV, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
#Reading the dataframe into the notebook
df = pd.read_csv("C:/Users/HP/Desktop/Projects/Datasets/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [ ]:
#printing the dataframe's first five rows of the dataset
df.head()

In [ ]:
#outputing the basic information about dataset
df.info()

In [ ]:
"""
This code shows the data type of columns in this dataset running this code will help me supervise and ensure that 
each column is assigned the appropriate data type based on the type of data it contains
"""
df.dtypes

# Data cleaning

In [ ]:
#Dropping unneeded column
df.drop("customer_ID", axis = 1, inplace=True)

In [ ]:
'''
After analyzing this data set's information for each column and row, i found some error on the senior citizen's column and 
for uniformity and order i shall be converting the values of the Senior citizen column from "1", "0" to "yes", "no" and 
also converting it's data type to an object data type
'''



# Replacing the data for SeniorCitizen which automatically converts the column's dtype to obj
df["senior_citizen"] = df["senior_citizen"].replace({0: "No", 1: "Yes"})
df["senior_citizen"].unique()

In [ ]:
"""
The total_charges column not left behind contains numerical data but has an object datatype 
and as such we shall change it to a float64 data type
"""



#The total charges column was found having a wrong data type and as such I changed it to the appropriate type
df["total_charges"] = pd.to_numeric(df["total_charges"], errors = "coerce")
df["total_charges"].dtype


In [ ]:
#Checking for missing values in this dataset
df.isna().sum().sort_values()

The total column was found to have 11 miss values from 11 rows and as such i shall be investigating the reason behind this.

In [ ]:
#inspecting the column with missing values to detect causes and patterns
df[df["total_charges"].isna()].describe()

In [ ]:
#inspecting the column with missing values to detect causes and patterns
df[df["total_charges"].isna()]

After analyzing this data set and the rows with missing values i found out that these row all had a zero tenure meaning all customers registed under these rows spent zero time with the company and as such i shall be filling the missing data in the total charges column with zeros

In [ ]:
df["total_charges"] = df["total_charges"].fillna(0)

In [ ]:

#Re-checking for missing values in this dataset one more time to ensure our changes reflected in the dataset positively
df.isna().sum().sort_values()

# Exploratory Data Analysis

## How long do customers  stay with the company ?

In [ ]:
#Dividing the data into two groups based on their churn status
churned = df[df['churn'] == 'Yes']
non_churned = df[df['churn'] == 'No']

In [ ]:
avg_tenure_churn_yes = round(churned.tenure.mean(),2)

avg_tenure_churn_no = round(non_churned.tenure.mean(),2)

# Combine the result in a dataframe
results = pd.DataFrame({
    'Metric': ['Avg Tenure (Churn: Yes)', 'Avg Tenure (Churn: No)'],
    'Value': [avg_tenure_churn_yes, avg_tenure_churn_no]
})

# Plot the results using a bar chart
plt.figure(figsize=(10,5))
ax = sns.barplot(x='Metric', y='Value', data=results)
ax.set(xlabel='',ylabel='Tenure month')
plt.xticks(rotation=0, ha='center')
plt.title('Average tenure for Churned and Non-Churned Customers')

# Annotate the bar chart with values
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.2f'), 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha = 'center', va = 'center', 
                xytext = (0, 4), 
                textcoords = 'offset points')
plt.show()

## A visualization on the distribution of values among the categorical columns

In [ ]:
review = df[['gender', 'senior_citizen', 'partner', 'dependents',
       'phone_service', 'multiple_lines', 'internet_service',
       'online_security', 'online_backup', 'device_protection', 'tech_support',
       'streaming_tV', 'streaming_movies', 'contract', 'paperless_billing',
       'payment_method']]
for i in review.columns:
    print(df[i].value_counts(normalize = True).round(3))
    plt.figure(figsize=(6, 4))
    sns.barplot(x=review[i].value_counts().index, 
                y=review[i].value_counts()
               ).set(title=i, xlabel='Label', ylabel='Frequency')
    plt.xticks(rotation = 45)
    plt.show()

The Visualization above shows the distribution of values among the following columns:

1. Gender: Categorizes customers as male or female.


2. Senior Citizen: Indicates whether a customer is a senior citizen (Yes / No).


3. Partner: Indicates whether a customer has a partner (Yes / No).


4. Dependents: Indicates whether a customer has dependents (Yes / No).


5. Phone Service: Indicates whether a customer has a phone service plan (Yes / No).


6. Multiple Lines: Indicates whether a customer has multiple phone lines (No phone service / Yes / No).


7. Internet Service: Indicates whether a customer has an internet service plan (DSL / Fiber optic / No).


8. Online Security: Indicates whether a customer subscribes to online security services (Yes / No / No internet service).


9. Online Backup: Indicates whether a customer subscribes to online backup services (Yes / No / No internet service).


10. Device Protection: Indicates whether a customer subscribes to device protection services (Yes / No / No internet service).


11. Tech Support: Indicates whether a customer utilizes technical support services (Yes / No / No internet service).


12. StreamingTV: Indicates whether a customer subscribes to a streaming TV service (Yes / No / No internet service).


13. Streaming Movies: Indicates whether a customer subscribes to a streaming movie service (Yes / No / No internet service).


14. Contract: Indicates whether a customer has a contract with the company (Month-to-month / One year / Two year).


15. Paperless Billing: Indicates whether a customer uses paperless billing (Yes / No).


16. Payment Method: Categorizes the customer's payment method (Electronic check / Mailed check / Bank transfer (automatic) /   Credit card (automatic)).





I used bar charts to visualize the distribution of unique values within each category and also show thier distribution via percentage among customers, this helps us understand the general composition of the customer base. 



This initial exploration provides a basic understanding of the data, the next steps will involve analyzing the churn rate.

## What is the distribution of churn among customers(gender based)?

In [ ]:
gender_dis = df["gender"].value_counts(normalize = True).round(3)
gender_dis

In [ ]:
#Data visualization on churn analysis
ax = sns.countplot(
    x = "gender",
    data = df
)
label = df['gender'].value_counts().apply(lambda x: x*100.0/len(df)).round(2).astype('str') + '%'
for p in ax.patches:
    ax.annotate(format(p.get_height()*100/len(df), '.2f'), 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha = 'center', va = 'center', 
                xytext = (0, 4), 
                textcoords = 'offset points')
plt.show()

Analysis of this data shows that 50 percentage of customers are males and 49 percentage of customers are females now let's analyze churn distribution among them 

In [ ]:
#analysis
df.groupby("gender")["churn"].value_counts().to_frame()

In [ ]:
#Data visualization on churn analysis
sns.countplot(
    x = "gender",
    hue = "churn",
    data = df
)
plt.show()

The data analysis above shows that Majority of customers who churn are Women with a count of 939 female customers who churned out of 3488 female customers

## What percentage of customers are senior Citizens?

In [ ]:
sns.countplot(
    x = df["senior_citizen"]
)
plt.show()

In [ ]:
df["senior_citizen"].value_counts(normalize = True)

Data analysis show that 16% out of 100% of our customers are Senior citizens while 84% of customers are Junior citizens

## What is the distribution of churn among Citizen class?

In [ ]:
df.groupby("senior_citizen")["churn"].value_counts().to_frame()

In [ ]:
df.groupby("senior_citizen")["churn"].value_counts(normalize = True).round(2).to_frame()

In [ ]:
#Data visualization on churn analysis
sns.countplot(
    x = "senior_citizen",
    hue = "churn",
    data = df
)
plt.show()

From the data analysis above insights has been gathered that
1. 76% of customers from the junior citizen class didn't churn while 24% of customers from the junior class did churn.
2. 58% of customers from the senior citiizen class didn't churn while 42% of customers from this class of citizens did churn.

This implies that services provided by the company is much more enjoyed by customers from the junior citizen class. 

## What is the distribution of churn among customers with partners?

In [ ]:
df["partner"].value_counts(normalize = True).round(2).to_frame()

This shows that 52% of customers are living without partners while 48% of customers live with partners 

In [ ]:
sns.countplot(
    x = df["partner"],
    hue = df["churn"]
)
plt.show()

In [ ]:
df.groupby("partner")["churn"].value_counts(normalize = True).to_frame()

Customers without partners have a higher distribution of churn as compared to customers with partners indicating this could be that customers who share the bills last longer

## What is the distribution of churn among customers with partners based on gender?

In [ ]:
df.groupby(["gender", "partner"])["churn"].value_counts(normalize = True).round(2).to_frame()

from the statistical analysis above it is seen that both male and  female customers with partners have a lower distribution of churn rate as compared to customers without partners 


## what is the distribution of churn among customers based on dependents?

In [ ]:
df["dependents"].value_counts(normalize = True)

In [ ]:
sns.countplot(
    x = "dependents",
    data = df
)
plt.show()

shown in the data analysis above 70% of customers are not dependent while 30% are dependent 

In [ ]:
df.groupby("dependents")["churn"].value_counts( normalize =True).to_frame()

In [ ]:
sns.countplot(
    x = "dependents",
    hue = "churn",
    data = df
)
plt.show()

Data analysis shows that dependent customers have the highest churn rate as compared to  undependent customers

## What is the churn rate of customers based on if they use a phone service?

In [ ]:
df.phone_service.value_counts(normalize = True).round(3).to_frame()


In [ ]:
sns.histplot(
    x = "phone_service",
    data = df
)
plt.show()

from the analysis above we can see that 90% of customers subscribe to a phone service while just 10% percentage of customers donot have a phone service subscription now let's run analysis to know the churn rate between customers who do and do not have a phone service subscription.

In [ ]:
df.groupby('phone_service')["churn"].value_counts(normalize = True).round(2).to_frame()

In [ ]:
sns.countplot(
    x ='phone_service',
    hue = "churn",
    data = df
    )
plt.xticks(rotation = 45)
plt.show()

As seen in the visualization and statistics above the rate of churn among customers with or without a phone service subcription is very low as majority of them do not churn.

## What is the churn rate of customers based on they number of lines the own?

In [ ]:
df["multiple_lines"].value_counts(normalize = True).round(2).to_frame()

In [ ]:
sns.histplot(
    df["multiple_lines"]
)
plt.show()

Insight gathered from the analysis above shows that out of the 90% of customers who made a phone service subscription 48% of them own multiple lines while 42% own just one lines and as usual 10% of customers did not make a phone service subscription 

In [ ]:
df.groupby('multiple_lines')["churn"].value_counts(normalize = True).round(2).to_frame()

In [ ]:
sns.countplot(
    x = 'multiple_lines',
    hue = "churn",
    data = df
    )
plt.xticks(rotation = 45)
plt.show()

The churn rate among customers who have multiple line is higher as compared to those with just a line and those without a phone service

## What is the churn rate of customers on the basis of the the use an internet service?

In [ ]:
df["internet_service"].value_counts(normalize = True).round(2).to_frame()

In [ ]:
sns.histplot(
    df["internet_service"]
)
plt.show()

This shows that 78 percentage of customers have a subcription for internet services while 22 percentage of customers do not have this subcription 

Among which 44 percentage of them use a fiber optic which is the type of internet service most customers use, while 34 percentage of them use DSL 


In [ ]:
df.groupby('internet_service')["churn"].value_counts(normalize = True).round(2).to_frame()

In [ ]:
sns.countplot(
    x = 'internet_service',
    hue = "churn",
    data = df
    )
plt.xticks(rotation = 45)
plt.show()

a high number of customers who use the fiber optic type of internet service churn does this mean that the service provide is not worth it? for out of a 100% of customers who use this service 42% of them turn to churn.
Also it is seen that customers with no internet subscription remain loyal to the company as a litte percentage of 7 churns out of a 100% OF THEM and the customers who use the DSL type of internet service have a churn rate of 19%.

## What is the churn Rate of customers on the basis of Online security?

In [ ]:
df["online_security"].value_counts(normalize = True).round(2).to_frame()


In [ ]:
sns.histplot(
    df["online_security"]
)
plt.show()

78 percentage of customers have a subscripiton for internet service and 22 percentage of customers don't, out of the 78 percentage of customers who have a subscribed tariff for internet services just 28 percentage do have an online security from the company and a high rate of 50 percenatge do not have an online security provided by the company

In [ ]:
sns.countplot(
    x = 'online_security',
    hue = "churn",
    data = df
    )
plt.xticks(rotation = 45)
plt.show()

In [ ]:
df.groupby('online_security')["churn"].value_counts(normalize = True).round(2).to_frame()

There is a high rate of churn for customers without a online security subscription from the company does this mean that they are unaware that the company actually provides an online security to customers or they think the security provided is poor and as such churn to find other places?

There is a low churn rate for customers with an online security subscription this shows that majority of them enjoy the services.

## What is the churn rate of customers on the basis of online backup?

In [ ]:
df["online_backup"].value_counts(normalize = True).round(2).to_frame()

In [ ]:
sns.histplot(
    df["online_backup"]
)
plt.show()

44% is a poor rate for customers who do not use an online backup service out of the 78% who make use of online services leaving jsut 34% of them who make use of this service

In [ ]:
sns.countplot(
    x = 'online_backup',
    hue = "churn",
    data = df
    )
plt.xticks(rotation = 45)
plt.show()

In [ ]:
df.groupby('online_backup')["churn"].value_counts(normalize = True).round(2).to_frame()

Analysis shows that more them three quarter of customers who use the online backup service do not churn and 40% of customers who do not use this servise churn and 93% out of the remaining 22% of customers who do not use an internet service do noot churn too

## What is the churn rate of customers on the basis of device protection?

In [ ]:
df["device_protection"].value_counts(normalize = True).round(2).to_frame()

In [ ]:
sns.histplot(
    df["device_protection"]
)
plt.show()

From the analysis ran above it can be seen that most customers (44% out of 78% who use internet service) do not use a device protection service, while a small rate of 34% use this service.

In [ ]:
df.groupby('device_protection')["churn"].value_counts(normalize = True).round(2).to_frame()

In [ ]:
sns.countplot(
    x = 'device_protection',
    hue = "churn",
    data = df
    )
plt.xticks(rotation = 45)
plt.show()

Critical analysis show that 39% of customers who do not use the device protection service provided by the company churn this might be a sign that the company needs step up thier device protection adverts format to customers as these customers may not have uused this serivce and churn due to ignorance as it is seen that a good rate of 77% of customers who use this service remain loyal to the company.

## What is the churn analysis of customers on the basis of Tech Support

In [ ]:
df["tech_support"].value_counts(normalize = True).round(2).to_frame()

In [ ]:
sns.histplot(
    df["tech_support"]
)
plt.show()

Statistics show that out of 78% of customers who use the internet 49% of them use the Tech support while 29% of them do not use the Tech support 

In [ ]:
df.groupby('tech_support')["churn"].value_counts(normalize = True).round(2).to_frame()

In [ ]:
sns.countplot(
    x = 'tech_support',
    hue = "churn",
    data = df
    )
plt.xticks(rotation = 45)
plt.show()

Analysis show that customers who do not use a tech support have a churn rate of 42% which can be a huge set back for the company. Also it is seen in this analysis that there is a low rate of 15% percentage for churn among customers who use a tech support.

## What's the churn rate among customers who maded made subscription to a streaming TV?

In [ ]:
df["streaming_tV"].value_counts(normalize = True).round(2).to_frame()

In [ ]:
sns.histplot(
    df["streaming_tV"]
)
plt.show()

The analysis and statistics above show that 40% out of 78% of customers who use an internet service have a subscription for a streaming TV service and 38% of customers do not have a streaming TV subscription.

In [ ]:
df.groupby('streaming_tV')["churn"].value_counts(normalize = True).round(2).to_frame()

In [ ]:
sns.countplot(
    x = 'streaming_tV',
    hue = "churn",
    data = df
    )
plt.xticks(rotation = 45)
plt.show()

analysing the streaming tv column by churn it is seen that 34% of customers who do not have this subscription churn while 66% OF them do not also as seen in the visualization and statistics above 70% of customers who subscribed to a streaming TV didn't churn while 30% of them did churn

## What is the churn rate among customers with a streaming Movies subscription?

In [ ]:
df["streaming_movies"].value_counts(normalize = True).round(3).to_frame()

In [ ]:
sns.histplot(
    df["streaming_movies"]
)
plt.show()

The analysis and statistics above show that 40% out of 78% of customers who use an internet service have a subscription for a streaming movies service and 38% of customers do not have a streaming movies subscription.

In [ ]:
df.groupby('streaming_movies')["churn"].value_counts(normalize = True).round(2).to_frame()

In [ ]:
sns.countplot(
    x = 'streaming_movies',
    hue = "churn",
    data = df
    )
plt.xticks(rotation = 45)
plt.show()

Now analyzing the churn rate it is seen from the statistics and visualization above that out of the 38% percentage of customers who do not have a streaming movies subscripiton, 34% of them churn while 66% don't also analyzing for those with this subscription, there is a 70% non churn rate and a 30% churn rate 

## What is the churn rate for customers based on their contract type?

In [ ]:
df["contract"].value_counts(normalize = True).round(2).to_frame()

In [ ]:
sns.histplot(
    df["contract"]
)
plt.show()

Based on the above statistics and visualization it is seen that; 

The contract for 55% of customers is month-to-month,

The contract for 24% of customers is two years and 

The contract for 21% of customers is one year

In [ ]:
sns.countplot(
    x = 'contract',
    hue = "churn",
    data = df
    )
plt.xticks(rotation = 45)
plt.show()

In [ ]:
df.groupby('contract')["churn"].value_counts(normalize = True).round(2).to_frame()

Now for the analysis of churn based on the type of contract a customer uses it is seen that the contract with the highest 

churn rate from customers is the month to month contract category where majority of the company customers fall and the contract 

category with the least churn rate and highest non churn rate is the two years contract which is the category that comes second in number of customers.

# What is the churn rate of customers on the basis of if or not the use a paperless bill?

In [ ]:
df["paperless_billing"].value_counts(normalize = True).round(2).to_frame()

In [ ]:
sns.histplot(
    df["paperless_billing"]
)
plt.show()

From this analysis and visualization seen above majority of customers use a paperless bill

In [ ]:
df.groupby('paperless_billing')["churn"].value_counts(normalize = True).round(2).to_frame()

In [ ]:
sns.countplot(
    x = 'paperless_billing',
    hue = "churn",
    data = df
    )
plt.xticks(rotation = 45)
plt.show()

it seems customers prefer bills in the traditional manner rather than using a paperless bill evidence in the analysis of churn versus paperless billing it is seen that customers who do not use a paperless bill have a higer non churn rate as compared to customers who use a paperless bill 

## What is the churn rate among customers based on thier prefered payment method?

In [ ]:
df["payment_method"].value_counts().to_frame()

In [ ]:
df["payment_method"].value_counts(normalize = True).round(2).to_frame()

In [ ]:
sns.histplot(
    df["payment_method"]
)
plt.xticks(rotation = 45)
plt.show()

As analyzed it seen that majority of customers used an electronic check for payments and the second most used payment method is the mailded check followed by the Automatic Bank transfer and the least used is the Automatic Credit card payment method

In [ ]:
df.groupby('payment_method')["churn"].value_counts().to_frame()

In [ ]:
df.groupby('payment_method')["churn"].value_counts(normalize = True).round(2).to_frame()

In [ ]:
sns.countplot(
    x = 'payment_method',
    hue = "churn",
    data = df
    )
plt.xticks(rotation = 45)
plt.show()

The Automatic Credit card payment method has the lowest churn rate and the highest non Churn rate  followed by the automatic Bank transfer, the Mailed check and lastly the Electronic Check

## What is the distribution of monthly and total charges among customer?

In [ ]:
df[["monthly_charges", "total_charges", "tenure"]].hist(figsize = (10,5))
plt.show()

# Model training 

## Preprocessing data

In [ ]:
df = pd.get_dummies(df, 
                        columns = ['gender', 'senior_citizen', 'partner', 'dependents', 'tenure',
       'phone_service', 'multiple_lines', 'internet_service',
       'online_security', 'online_backup', 'device_protection', 'tech_support',
       'streaming_tV', 'streaming_movies', 'contract', 'paperless_billing',
       'payment_method', 'monthly_charges', 'total_charges'],
                        drop_first = True)

In [ ]:
df["churn"] = df['churn'].replace({"Yes": 1, "No": 0})

In [ ]:
df.head()

## Feature variable 

In [ ]:
x = df.drop("churn", axis = 1)

## Target variable

In [ ]:
y = df["churn"]

## Splitting data set for training and testing

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

# KNeighbors Classifier

## Hyperparameter tuning

In [ ]:
#kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

In [ ]:
'''param_grid = {
    "n_neighbors": np.arange(2,10),
    "metric" : ['euclidean', 'manhattan']
}'''

In [157]:
#instantiating model
knn = KNeighborsClassifier(n_neighbors = 3)

In [158]:
#tunning = RandomizedSearchCV(knn, param_grid, cv = kf, n_iter = 2)

In [159]:
#tunning.fit(x_train, y_train)


In [160]:
#print(tunning.best_params_, tunning.best_score_)

The lines of codes for hyperparameter tunning above are computationally expensive and as such after getting the best params from the output, i turned them into comments to optimize the run time for the rest of the codes

In [161]:
#Fitting training data to model 
knn.fit(x_train, y_train) 

KNeighborsClassifier(n_neighbors=3)

In [162]:
#Using the training data to predict the target variable
y_train_pred = knn.predict(x_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [163]:
#Evaluating the Knerighbor classifier model's performance
accuracy_score(y_train, y_train_pred)


0.8549695740365112

In [164]:
confused_matrix_train = confusion_matrix(y_train, y_train_pred)
print(confused_matrix_train)

[[3287  348]
 [ 367  928]]


In [ ]:
sns.heatmap(
    confused_matrix_train, 
    annot = True, 
    cmap ="viridis",
    fmt = 'g',
    xticklabels = ["Non-churn(0)","Churn(1)"],
    yticklabels = ["Non-churn(0)","Churn(1)"]
)
plt.title('Kneighbors Classifier confusion matrix')
plt.show()

In [ ]:
class_report = classification_report(y_train, y_train_pred)
print(class_report)

From the Classification report above:
1. The precision which is the is the number of true positives divided by the sum of all positive predictions,
a high precision means having a lower false positive rate.
2. The Recall which is the number of true positives divided by the sum of true positives and false negatives,
a high recall reflects a lower false negative rate
3. The F1-score is the harmonic mean of precision and recall.

In [ ]:
y_train_pred_prob = knn.predict_proba(x_train)[:,1]

In [ ]:
fpr, tpr, theshold = roc_curve(y_train, y_train_pred_prob)
plt.plot([0, 1], [0, 1], "k--")
print(roc_auc_score(y_train, y_train_pred_prob))
plt.plot(fpr, tpr)
plt.show()

In [154]:
y_test_pred = knn.predict(x_test_scaled)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [165]:
accuracy_score(y_test_pred, y_test)

0.7292948414576431

In [156]:
confused_matrix_test = confusion_matrix(y_test, y_test_pred)
print(confused_matrix_test)

[[1524   15]
 [ 557   17]]


In [ ]:
sns.heatmap(
    confused_matrix_test, 
    annot = True, 
    cmap ="turbo",
    fmt = 'g',
    xticklabels = ["Non-churn(0)","Churn(1)"],
    yticklabels = ["Non-churn(0)","Churn(1)"]
)
plt.title('Kneighbors Classifier confusion matrix')
plt.show()

In [ ]:
class_report = classification_report(y_test, y_test_pred)
print(class_report)

In [ ]:
y_test_pred_prob = knn.predict_proba(x_test)[:,1]

In [ ]:
fpr, tpr, theshold = roc_curve(y_test, y_test_pred_prob)
plt.plot([0, 1], [0, 1], "k--")
print(roc_auc_score(y_test, y_test_pred_prob))
plt.plot(fpr, tpr)
plt.show()

# Logistic Regression`

In [ ]:
#instantiating model
log = LogisticRegression(max_iter = 1000)

In [ ]:
log.fit(x_train, y_train)

In [ ]:
log_y_pred_train = log.predict(x_train)

In [ ]:
accuracy_score(y_train, log_y_pred_train)

In [ ]:
log_train_confused = confusion_matrix(y_train, log_y_pred_train)
print(log_train_confused)

In [ ]:
sns.heatmap(
    log_train_confused,
    annot = True, 
    cmap ='viridis',
    fmt = 'g',
    xticklabels = ["Non-churn(0)","Churn(1)"],
    yticklabels = ["Non-churn(0)","Churn(1)"]
)
plt.show()

In [ ]:
log_class_report = classification_report(y_train, log_y_pred_train)
print(log_class_report)

In [ ]:
log_y_train_pred_prob = log.predict_proba(x_train)[:,1]

In [ ]:
fpr, tpr, theshold = roc_curve(y_train, log_y_train_pred_prob)
plt.plot([0, 1], [0, 1], "k--")
print(roc_auc_score(y_train, log_y_train_pred_prob))
plt.plot(fpr, tpr)
plt.show()

In [ ]:
log_y_pred_test = log.predict(x_test)

In [ ]:
accuracy_score(log_y_pred_test, y_test)

In [ ]:
log_confused_test = confusion_matrix(log_y_pred_test, y_test)
print(log_confused_test)

In [ ]:
sns.heatmap(
    log_confused_test,
    annot = True,
    cmap ='turbo',
    fmt = 'g',
    xticklabels = ["Non-churn(0)","Churn(1)"],
    yticklabels = ["Non-churn(0)","Churn(1)"]
)
plt.show()

In [ ]:
log_test_proba = log.predict_proba(x_test)[:,1]

In [ ]:
fpr, tpr, threshold = roc_curve(y_test, log_test_proba)
plt.plot([0, 1], [0, 1], "k--")
print(roc_auc_score(y_test, log_test_proba))
plt.plot(fpr, tpr)
plt.show()

# Decision Tree Classifier

In [ ]:
dt = DecisionTreeClassifier(criterion = "gini", max_depth = 3, random_state = 42)

In [ ]:
dt.fit(x_train, y_train)

In [ ]:
dt_train_pred = dt.predict(x_train)

In [ ]:
accuracy_score(dt_train_pred, y_train)

In [ ]:
dt_train_confused = confusion_matrix(dt_train_pred, y_train)
print(dt_train_confused)

In [ ]:
sns.heatmap(
    dt_train_confused,
    fmt = "g",
    cmap = "viridis",
    annot = True,
    xticklabels = ["Non-churn(0)","Churn(1)"],
    yticklabels = ["Non-churn(0)","Churn(1)"]
)
plt.show()

In [ ]:
print(classification_report(dt_train_pred, y_train))

In [ ]:
dt_train_proba = dt.predict_proba(x_train)[:,1]

In [ ]:
fpr, tpr, theshold = roc_curve(y_train, dt_train_proba)
plt.plot([0, 1], [0, 1], "k--")
print(roc_auc_score(y_train, dt_train_proba))
plt.plot(fpr, tpr)
plt.show()

In [ ]:
dt_test = dt.predict(x_test)

In [ ]:
accuracy_score(dt_test, y_test)

In [ ]:
dt_test_confused = confusion_matrix(dt_test, y_test)
print(dt_test_confused)

In [ ]:
sns.heatmap(
    dt_test_confused,
    fmt = 'g',
    annot = True,
    cmap = "turbo",
    xticklabels = ["Non-churn(0)","Churn(1)"],
    yticklabels = ["Non-churn(0)","Churn(1)"]
)
plt.show()

In [ ]:
print(classification_report(dt_test, y_test))

In [ ]:
dt_test_proba = dt.predict_proba(x_test)[:,1]

In [ ]:
fpr, tpr, threshold = roc_curve(y_test, dt_test_proba)
plt.plot([0, 1], [0, 1], "k--")
print(roc_auc_score(y_test, dt_test_proba))
plt.plot(fpr, tpr)
plt.show()